<a href="https://colab.research.google.com/github/Dinhthixuanbinh/ONNx_demo/blob/main/Pytorch__to_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install onnx
!pip install tensorflow-addons
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .
!pip install torchvision
!pip install onnx-tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.2 MB/s eta 0:00:00
Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 6516, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 6516 (delta 326), reused 383 (delta 261), pack-reused 6051
Receiving objects: 100% (6516/6516), 1.98 MiB | 2.72 MiB/s, done.
Resolving deltas: 100% (5051/5051), done.
Obtaining file:///content/onnx-tensorflow
  Preparing metadata (setup.py) ... done
  Running setup.py develop for onnx-tf
ERROR: Could not find a version that satisfies the requirement onnx-tensorflow (from versions: none)
ERROR: No matching distribution found for onnx-tensorflow


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import onnx
from onnx import backend


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} \tLoss: {:.6f}'.format(
                    epoch,  loss.item()))

In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)


torch.manual_seed(1)
device = torch.device("cuda")

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

100%|██████████| 9912422/9912422 [00:00<00:00, 257595841.64it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 44453465.62it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 71286090.92it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15757261.18it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Train Epoch: 0 	Loss: 2.327492

Test set: Average loss: 0.2007, Accuracy: 9421/10000 (94%)

Train Epoch: 1 	Loss: 0.416417

Test set: Average loss: 0.1218, Accuracy: 9621/10000 (96%)

Train Epoch: 2 	Loss: 0.215019

Test set: Average loss: 0.0978, Accuracy: 9689/10000 (97%)

Train Epoch: 3 	Loss: 0.409411

Test set: Average loss: 0.0831, Accuracy: 9748/10000 (97%)

Train Epoch: 4 	Loss: 0.178752

Test set: Average loss: 0.0750, Accuracy: 9766/10000 (98%)

Train Epoch: 5 	Loss: 0.120448

Test set: Average loss: 0.0665, Accuracy: 9784/10000 (98%)

Train Epoch: 6 	Loss: 0.194846

Test set: Average loss: 0.0601, Accuracy: 9816/10000 (98%)

Train Epoch: 7 	Loss: 0.272399

Test set: Average loss: 0.0580, Accuracy: 9820/10000 (98%)

Train Epoch: 8 	Loss: 0.201344

Test set: Average loss: 0.0545, Accuracy: 9819/10000 (98%)

Train Epoch: 9 	Loss: 0.088901

Test set: Average loss: 0.0530, Accuracy: 9846/10000 (98%)

Tra

In [9]:
torch.save(model.state_dict(), 'mnist.pth')


In [11]:
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))

dummy_input = Variable(torch.randn(1, 1, 28, 28))
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")


In [13]:
!pip install git+https://github.com/onnx/onnx-tensorflow.git


  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-emnqocur
  Running command git clone --filter=blob:none --quiet https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-emnqocur
  Resolved https://github.com/onnx/onnx-tensorflow.git to commit ee0c5e537b3cebbddc5773871e6786e6468c7f3f
  Preparing metadata (setup.py) ... done
  Created wheel for onnx-tf: filename=onnx_tf-1.10.0-py3-none-any.whl size=226641 sha256=786f201807548b2126e08528e47526bc8559927cf72f4440c2078c345c07c01a
  Stored in directory: /tmp/pip-ephem-wheel-cache-go_xi2mi/wheels/26/8d/09/2c2cfa55123a5633aae58e44c4c6baf908c7f2e75470e0f392
Successfully built onnx-tf
  Attempting uninstall: onnx-tf
    Found existing installation: onnx-tf 1.10.0
    Uninstalling onnx-tf-1.10.0:
      Successfully uninstalled onnx-tf-1.10.0


In [14]:
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('mnist.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
#
tf_rep.export_graph('mnist.pb')


In [ ]:
#
import numpy as np
from IPython.display import display
from PIL import Image




print('Image 1:')
img = Image.open('/content/img1.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as ', np.argmax(output))
print('------------------------------------------------------------------------------')
print('Image 2:')
img = Image.open('/content/img2.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as ', np.argmax(output))